# Gerekli Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Veri Okuma

In [ ]:
meta_data = pd.read_csv("/kaggle/input/titanic/train.csv")
data = meta_data.copy()
data.head()

In [ ]:
meta_test = pd.read_csv("/kaggle/input/titanic/test.csv")
test = meta_test.copy()
test.head()

In [ ]:
data.info()
print("\n")
test.info()

In [ ]:
print("Veri tipleri \n", data.info())
print("Verinin boyutu :",data.shape)
print("Verinin skalaları \n",data.describe())
print("Verideki boş satır sayıları \n", data.isnull().sum())

# Veri Temizleme ve Dönüştürme İşlemleri


## Çok fazla boş veri içeren verileri silme

In [ ]:
def drops(data):
  data.drop("Ticket", axis=1, inplace=True)
  data.drop("Cabin", axis=1, inplace=True)

In [ ]:
drops(data)
print(data.isnull().sum())
print("----")
drops(test)
print(test.isnull().sum())

In [ ]:
sns.catplot(data=data, x="Age", y="Embarked", hue="Sex", kind="boxen")

In [ ]:
sns.catplot(data=data, x="Age", y="Sex", kind="violin", color=".9", inner=None)
sns.swarmplot(data=data, x="Age", y="Sex", size=3)

In [ ]:
g = sns.catplot(
    data=data, x="Sex", y="Survived", col="Embarked",
    kind="bar", height=4, aspect=.6,
)
g.set_axis_labels("", "Survival Rate")
g.set_xticklabels(["Male", "Female"])
g.set_titles("{col_name} {col_var}")
g.set(ylim=(0, 1))
g.despine(left=True)

## Kategorik verilerin analize hazır hale getirilmesi

In [ ]:
data["Embarked"].value_counts()

In [ ]:
print("Embarked \n",data["Embarked"].unique())
print("Sex \n",data["Sex"].unique())

In [ ]:
data["Embarked"].fillna("S", inplace=True)
test["Fare"].fillna(test["Fare"].mean(),inplace=True)
print("Embarked \n",data["Embarked"].unique())

In [ ]:
test.isnull().sum()

In [ ]:
data.isnull().sum()

In [ ]:
def age_fill_(data):
  age_fill = data.groupby(['Pclass', 'Sex'])['Age'].mean()
  data['Age'] = data.apply(lambda x: age_fill[x['Pclass'], x['Sex']] if pd.isnull(x['Age']) else x['Age'], axis=1)
age_fill_(data)
age_fill_(test)

In [ ]:
test.isnull().sum()

In [ ]:
def convert_datas(data):
  for feature in ["Embarked", "Sex"]:
    new_cols = pd.get_dummies(data[feature])
    data = data.join(new_cols)
    data.drop(feature, axis=1, inplace=True)
  return data
data = convert_datas(data)
data.head(10)

In [ ]:
test = convert_datas(test)
test.head(10)

In [ ]:
print("Veri tipleri \n", data.info())
print("Verinin boyutu :",data.shape)
print("Verinin skalaları \n",data.describe())
print("Verideki boş satır sayıları \n", data.isnull().sum())

In [ ]:
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

In [ ]:
data['Title'] = data['Name'].apply(get_title)
data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don',                                                   'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data.drop('Name',axis=1,inplace=True)

In [ ]:
title_col = pd.get_dummies(data["Title"])
data = data.join(title_col)
data.drop("Title", axis=1, inplace=True)

In [ ]:
test['Title'] = test['Name'].apply(get_title)
test['Title'] = test['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don',
                                       'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')
test.drop('Name',axis=1,inplace=True)

In [ ]:
title_col = pd.get_dummies(test["Title"])
test = test.join(title_col)
test.drop("Title", axis=1, inplace=True)

In [ ]:
test

In [ ]:
data

## Ayrık verilerin temizlenmesi

In [ ]:
sns.boxplot(data=data["Age"])
plt.show()

In [ ]:
sns.boxplot(data=data["Fare"])
plt.show()

In [ ]:
def remove_outliers(data):#test
    indexs = []
    for feature in ["Age", "Fare"]:
        q1 = data[feature].quantile(0.25)
        q3 = data[feature].quantile(0.75)
        IQR = q3-q1
        under_line = q1-1.5*IQR
        high_line = q3+1.5*IQR
        discrete_min = data[data[feature]<under_line][feature]
        discrete_max = data[data[feature]>high_line][feature]
        discrete = pd.concat([discrete_min, discrete_max], axis=0).index
        for i in discrete:
            indexs.append(i)
    data = data.drop(data.index[indexs])
    #test = test.drop(test.index[indexs])
    data.index = range(len(data))
    #test.index = range(len(test))
    return data

In [ ]:
data = remove_outliers(data)

In [ ]:
sns.boxplot(data=data["Age"])
plt.show()

In [ ]:
sns.boxplot(data=data["Fare"])
plt.show()

In [ ]:
print(data)
print(test)

In [ ]:
corr = data.corr()
print(corr['Survived'].sort_values(ascending=False))

In [ ]:
f, ax = plt.subplots(figsize=(9,9))
sns.heatmap(data.corr(), annot=True, linewidth=5, ax=ax)

In [ ]:
data.hist(figsize=(18,12))
plt.show()

# Ayırma, model oluşturma işlemleri

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
X = data.drop("Survived", axis=1)
y = data["Survived"]
print(X)
print(y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)

In [ ]:
X_train_prediction = model.predict(X_train)
print(X_train_prediction)

In [ ]:
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of training data : ', training_data_accuracy)

In [ ]:
X_test_prediction = model.predict(X_test)

In [ ]:
print(X_test_prediction)

In [ ]:
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of test data : ', test_data_accuracy)